# Projet : Détection de Fake News via Pipeline RAG
#### Étape 1 — Prétraitement des articles
##### Objectif de ce notebook :
##### Nettoyer et préparer les articles pour la recherche et la génération.
##### On effectue ici les traitements de texte nécessaires avant la vectorisation
##### et le stockage dans la base de données vectorielle (ChromaDB).


In [2]:
# 1. Importations et configuration initiale
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings
import spacy
from tqdm import tqdm

# Ignorer certains warnings inutiles
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Charger le modèle spaCy pour la lemmatisation
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

# Définir les chemins de données
RAW_PATH = "data/raw"
PROCESSED_PATH = "data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

#### 🎯 Objectif
L’objectif de cette étape est de transformer les fichiers CSV d’articles (`True.csv`, `Fake.csv`)
en un dataset nettoyé, structuré et prêt à être vectorisé.

#### Étapes du pipeline :
1. Chargement et fusion des jeux de données (Fake / True)
2. Vérifications de base (valeurs manquantes, doublons)
3. Nettoyage des textes :
   - Suppression des balises HTML
   - Suppression des URLs
   - Suppression des caractères spéciaux
   - Normalisation des espaces
   - Mise en minuscules
4. Suppression des stopwords et lemmatisation (avec spaCy)
5. Découpage des articles en *chunks* (morceaux)
6. Sauvegarde des données nettoyées


In [ ]:
#3. Chargement des fichiers CSV
csv_files = [f for f in os.listdir(RAW_PATH) if f.endswith(".csv")]
print(f"Fichiers trouvés : {csv_files}")

dfs = []
for file_name in csv_files:
    df_tmp = pd.read_csv(os.path.join(RAW_PATH, file_name))
    # Ajouter une colonne label en fonction du nom du fichier
    df_tmp["label"] = "Fake" if "fake" in file_name.lower() else "True"
    dfs.append(df_tmp)

# Fusionner les DataFrames
df = pd.concat(dfs, ignore_index=True)
print(f"📊 Taille totale du dataset : {df.shape}")
df.head()


Fichiers trouvés : ['True.csv', 'Fake.csv']
📊 Taille totale du dataset : (44898, 5)


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True


In [4]:
#4. Vérification des données
print("Valeurs manquantes par colonne :")
print(df.isna().sum())

print("\nNombre de doublons :", df.duplicated().sum())

# Suppression des doublons et des valeurs manquantes
df = df.drop_duplicates().fillna("")


Valeurs manquantes par colonne :
title      0
text       0
subject    0
date       0
label      0
dtype: int64

Nombre de doublons : 209


In [5]:
#5. Nettoyage des textes
def clean_text(text):
    """
    Nettoie un texte :
    - Supprime les balises HTML
    - Supprime les URLs
    - Supprime les caractères spéciaux
    - Supprime les espaces multiples
    - Convertit en minuscules
    """
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^A-Za-z0-9À-ÿ\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

# Vérification du nettoyage
df[["text", "clean_text"]].head(3)


,text,clean_text
0,WASHINGTON (Reuters) - The head of a conservat...,washington reuters the head of a conservative ...
1,WASHINGTON (Reuters) - Transgender people will...,washington reuters transgender people will be ...
2,WASHINGTON (Reuters) - The special counsel inv...,washington reuters the special counsel investi...


In [ ]:
#6. Suppression des stopwords et lemmatisation
from spacy.lang.en.stop_words import STOP_WORDS

def lemmatize_text(text):
    """Applique la lemmatisation et retire les stopwords"""
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text.lower() not in STOP_WORDS]
    return " ".join(tokens)

# Pour accélérer le test, on peut le faire sur un échantillon d'abord :
sample = df["clean_text"].iloc[:3].apply(lemmatize_text)
print(sample)

In [6]:
# 7. Découpage en chunks
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Découpe un texte en morceaux de longueur fixe (en mots)
    avec un chevauchement pour conserver le contexte.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Exemple
example_text = df["clean_text"].iloc[0]
print(chunk_text(example_text, chunk_size=50, overlap=10)[:2])


['washington reuters the head of a conservative republican faction in the u s congress who voted this month for a huge expansion of the national debt to pay for tax cuts called himself a fiscal conservative on sunday and urged budget restraint in 2018 in keeping with a sharp pivot', 'budget restraint in 2018 in keeping with a sharp pivot under way among republicans u s representative mark meadows speaking on cbs face the nation drew a hard line on federal spending which lawmakers are bracing to do battle over in january when they return from the holidays on wednesday']


In [7]:
#8. Application du découpage à tout le dataset
all_chunks = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    chunks = chunk_text(row["clean_text"], chunk_size=200, overlap=50)
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "article_id": idx,
            "title": row.get("title", ""),
            "chunk_id": i,
            "chunk_text": chunk,
            "label": row["label"]
        })

df_chunks = pd.DataFrame(all_chunks)
print(f"✅ Nombre total de chunks : {df_chunks.shape[0]}")
df_chunks.head()


100%|██████████| 44689/44689 [00:04<00:00, 9819.05it/s] 


✅ Nombre total de chunks : 145486


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True
2,0,"As U.S. budget fight looms, Republicans flip t...",2,responsibility democratic u s representative j...,True
3,0,"As U.S. budget fight looms, Republicans flip t...",3,mean food stamps housing assistance medicare a...,True
4,0,"As U.S. budget fight looms, Republicans flip t...",4,from deportation and provides them with work p...,True


In [ ]:
#9. Sauvegarde des fichiers nettoyés 
# Sauvegarde du dataset nettoyé complet
cleaned_path = os.path.join(PROCESSED_PATH, "articles_cleaned.csv")
df.to_csv(cleaned_path, index=False)

# Sauvegarde du dataset chunké
chunks_path = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks.to_csv(chunks_path, index=False)

print(f" Données nettoyées sauvegardées : {cleaned_path}")
print(f" Chunks sauvegardés : {chunks_path}")


📂 Données nettoyées sauvegardées : data/processed/articles_cleaned.csv
📂 Chunks sauvegardés : data/processed/articles_chunks.csv


In [9]:
#10. Tests simples 
# Vérifier qu’il n’y a pas de texte vide
assert (df_chunks["chunk_text"].str.strip() != "").all(), " Des chunks vides détectés !"

# Vérifier la cohérence du label
assert set(df_chunks["label"].unique()) <= {"True", "Fake"}, "Label invalide détecté !"

print("Tous les tests basiques sont passés.")

Tous les tests basiques sont passés.


#### ✅ Étapes réalisées :
- Chargement et fusion des fichiers True/Fake
- Nettoyage complet des textes (HTML, URLs, caractères spéciaux, etc.)
- Mise en minuscules
- Lemmatisation et suppression des stopwords (optionnelle)
- Découpage en chunks pour la vectorisation
- Sauvegarde des jeux de données nettoyés et prêts pour ChromaDB

#### 🚀 Étape suivante :
👉 Passer à la **vectorisation et au stockage dans ChromaDB**
où chaque chunk sera converti en vecteur à l’aide du modèle d’embedding (Ollama).


### Étape 2 : Vectorisation et stockage dans ChromaDB
#### Objectif :
##### Charger les chunks nettoyés, créer une collection dans ChromaDB,
##### générer les embeddings via Ollama et stocker le tout avec les métadonnées.

In [48]:
# 1 Installation des bibliothèques
# #!pip install chromadb ollama --quiet
import chromadb
from chromadb.utils import embedding_functions

In [49]:
#2. Chargement du dataset chunké
CHUNKS_PATH = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks = pd.read_csv(CHUNKS_PATH)
print(f"{df_chunks.shape[0]} chunks chargés depuis {CHUNKS_PATH}")
df_chunks.head(2)

145486 chunks chargés depuis data/processed/articles_chunks.csv


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True


In [51]:
#3. Initialisation du client ChromaDB et création de la collection

# Création (ou récupération) d’un client local
client = chromadb.Client()

# Nom de la collection
collection_name = "fake_news_chunks"

# Supprimer la collection si elle existe déjà (pour test propre)
if collection_name in [c.name for c in client.list_collections()]:
    client.delete_collection(name=collection_name)

# Créer la collection
collection = client.create_collection(name=collection_name)
print(f" Collection '{collection_name}' créée avec succès.")

 Collection 'fake_news_chunks' créée avec succès.


In [57]:
# 4. Définir la fonction d’embedding avec Ollama
#rq : Assure  d’avoir lancé le serveur Ollama en parallèle : Terminal → ollama serve
embedding_fn = embedding_functions.OllamaEmbeddingFunction(
    model_name="all-minilm"  # modèle léger conseillé
)

In [58]:
# 5. Sélection d’un petit échantillon (pour test rapide)

# Pour éviter un traitement trop long au début
sample_df = df_chunks.sample(10, random_state=42).reset_index(drop=True)
sample_df.head(2)

,article_id,title,chunk_id,chunk_text,label
0,26204,WATCH: One Of Trump’s Biggest Supporters Is O...,3,supporting donald trump for president for too ...,Fake
1,22451,Joe Biden Is Downright Ruthless When Rich ‘A*...,0,joe biden was speaking at the skybridge altern...,Fake


In [59]:
# 6. Ajout des chunks dans ChromaDB
for idx, row in sample_df.iterrows():
    chunk_id = f"{row['article_id']}_{row['chunk_id']}"
    metadata = {
        "title": row.get("title", ""),
        "label": row["label"],
        "article_id": int(row["article_id"]),
    }

    collection.add(
        ids=[chunk_id],
        documents=[row["chunk_text"]],
        metadatas=[metadata],
        embeddings=embedding_fn([row["chunk_text"]])
    )

print(f" {len(sample_df)} chunks ajoutés à la collection '{collection_name}'.")


 10 chunks ajoutés à la collection 'fake_news_chunks'.


In [64]:
# 7. Vérification du stockage
results = collection.peek()
print(" Exemple de document stocké :")
results

 Exemple de document stocké :


{'ids': ['26204_3',
  '22451_0',
  '16967_3',
  '39130_2',
  '44364_0',
  '31827_11',
  '32417_2',
  '7251_1',
  '21995_0',
  '10910_0'],
 'embeddings': array([[ 0.03148066,  0.06609543, -0.00260025, ..., -0.01345606,
         -0.00472292,  0.00255966],
        [-0.04013475, -0.01899961,  0.02693708, ...,  0.00223051,
         -0.12071282,  0.03988767],
        [-0.00988505,  0.04822049,  0.02915613, ...,  0.01965448,
         -0.03194366,  0.02829264],
        ...,
        [ 0.04085801, -0.00852068,  0.0645113 , ..., -0.05173074,
          0.04013757, -0.03626084],
        [-0.01746017, -0.0362521 ,  0.10753163, ..., -0.0122874 ,
          0.00103011, -0.02215472],
        [-0.0513751 , -0.01095264,  0.01819525, ..., -0.09877898,
          0.01238176,  0.09949215]], shape=(10, 384)),
 'documents': ['supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough so burns literally played the race card

In [62]:
# 8. Test rapide de recherche sémantique
query = "The president made a false statement about the election"
results = collection.query(query_texts=[query], n_results=3)

print(" Résultats de la recherche sémantique :")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"- [{meta['label']}] {doc[:150]}...\n")

 Résultats de la recherche sémantique :
- [Fake] s leaks to confabulate that russian s hacked the dnc to influence the elections is the claim of one well known russian spy then 17 u s intelligence ag...

- [Fake] supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough s...

- [Fake] joe biden was speaking at the skybridge alternatives salt conference when one rich a hole decided to disrespect the former vice president s dead son b...



In [63]:
# 9. Sauvegarde et fin de l’étape 2
print("""
Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification
""")


Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification

